# seq2seqレプリケーション

seq2seqで結果を出していく

In [1]:
import torch
import random
import numpy as np
import torch.optim as optim
import torch.nn as nn
import transformers 
import time

/Users/akihito/.local/share/virtualenvs/CodeBERT-jLTtsSbs/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
# データロード
patterns = ["train","valid","test"]
frases = ["buggy","fixed"]
filename = ".buggy-fixed."
RelativePath = "data/codereview/"
def getPath(pattern_idx: int, frases_idx: int) -> str:
    return RelativePath + patterns[pattern_idx] + filename + frases[frases_idx]

In [5]:
# データのロード

def loadData():
    datas = {}
    for i in range(len(patterns)):
        datas[patterns[i]] = {}
        for j in range(len(frases)):
            path = getPath(i,j)
            with open(path, "r") as f:
                lines = f.readlines()
                lines = [line.split(" ") for line in lines]
                datas[patterns[i]][frases[j]] = lines
    return datas
datas = loadData()

In [6]:
from torch.utils.data import DataLoader

In [7]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data: list, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.tokenizer(self.data[idx])

In [8]:
from collections import Counter
from torchtext.vocab import Vocab
import io


In [9]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, PreTrainedTokenizer, AutoTokenizer

tokenizer = lambda x:  x.split(" ") # 今回はただのスペース区切り

In [10]:

train_filepaths = [getPath(0,0), getPath(0,1)]
val_filepaths = [getPath(1,0), getPath(1,1)]
test_filepaths = [getPath(2,0), getPath(2,1)]

train_filepaths

['data/codereview/train.buggy-fixed.buggy',
 'data/codereview/train.buggy-fixed.fixed']

In [11]:
# https://pytorch.org/text/stable/vocab.html
# https://gist.github.com/nariaki3551/cca40b4011c3b656df9cb1fa487612a7
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter, OrderedDict
from torchtext.utils import download_from_url, extract_archive
import io



init_token = "<sos>"
eos_token ="<eos>"
padding_token  = "<pad>"

def build_vocab(filepath, tokenizer):
  counter = Counter()
  with io.open(filepath, encoding="utf8") as f:
    for i, string_ in enumerate(f):
      counter.update(tokenizer(string_))
      # if i == 1000 : break 
  sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
  ordered_dict = OrderedDict(sorted_by_freq_tuples)
  
  vocab = torchtext.vocab.vocab(ordered_dict,specials=[init_token, eos_token, padding_token])
  
  default_index = -1
  vocab.set_default_index(default_index)
  return vocab
    

vocab = build_vocab(train_filepaths[0],tokenizer )
print(vocab.get_itos()[0:5])
print(len(vocab))


['<sos>', '<eos>', '<pad>', '(', ')']
496


In [12]:
# dataのtokenize
def data_process(filepaths):
  raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))
  raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
  data = []
  for (raw_de, raw_en) in zip(raw_de_iter, raw_en_iter):
    de_tensor_ = torch.tensor([vocab[token] for token in tokenizer(raw_de)],
                            dtype=torch.long)
    en_tensor_ = torch.tensor([vocab[token] for token in tokenizer(raw_en)],
                            dtype=torch.long)
    data.append((de_tensor_, de_tensor_))
  return data

train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)
np.shape(test_data)


/Users/akihito/.local/share/virtualenvs/CodeBERT-jLTtsSbs/lib/python3.9/site-packages/numpy/core/fromnumeric.py:2009: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  result = asarray(a).shape
/Users/akihito/.local/share/virtualenvs/CodeBERT-jLTtsSbs/lib/python3.9/site-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarr

(1719, 2)

In [13]:
# データローダーの作成
import torch


BATCH_SIZE = 32
SOS_IDX = vocab['<sos>']
EOS_IDX = vocab['<eos>']
PAD_IDX = vocab["<pad>"]

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def generate_batch(data_batch):
  src_batch, trg_batch = [], []
  for (src_item, trg_item) in data_batch:
    src_batch.append(torch.cat([torch.tensor([SOS_IDX]), src_item, torch.tensor([EOS_IDX])], dim=0))
    trg_batch.append(torch.cat([torch.tensor([SOS_IDX]), trg_item, torch.tensor([EOS_IDX])], dim=0))
  src_batch = pad_sequence(src_batch, padding_value=PAD_IDX) #文字列が一定になるようにトークンで埋める（validとtestのみ）
  trg_batch = pad_sequence(trg_batch, padding_value=PAD_IDX)
  return src_batch, trg_batch

train_iter = DataLoader(train_data, 
                        batch_size=BATCH_SIZE,
                        collate_fn=generate_batch,
                        shuffle=False
                      )
valid_iter = DataLoader(val_data,
                        batch_size=BATCH_SIZE,
                        collate_fn=generate_batch,
                        shuffle=False, 
                        )
test_iter = DataLoader(test_data,
                       batch_size=BATCH_SIZE,
                       shuffle=False, 
                       collate_fn=generate_batch)

In [14]:
for i,v in enumerate(test_iter):
    print(i)
    print(np.shape(v[0]))
    print(v[0])

0
torch.Size([99, 32])
tensor([[  0,   0,   0,  ...,   0,   0,   0],
        [ 42,  20,  42,  ...,  42,  20,  20],
        [ 51, 146,  30,  ...,  44,  51,  11],
        ...,
        [  2,   2,   2,  ...,   2,   2,   2],
        [  2,   2,   2,  ...,   2,   2,   2],
        [  2,   2,   2,  ...,   2,   2,   2]])
1
torch.Size([97, 32])
tensor([[  0,   0,   0,  ...,   0,   0,   0],
        [ 42,  20,  20,  ...,  20,  20,  20],
        [ 51,  11, 327,  ...,  30,  30,  24],
        ...,
        [  2,   2,   2,  ...,   2,   2,   2],
        [  2,   2,   2,  ...,   2,   2,   2],
        [  2,   2,   2,  ...,   2,   2,   2]])
2
torch.Size([100, 32])
tensor([[  0,   0,   0,  ...,   0,   0,   0],
        [ 20,  20,  71,  ...,  20,  20,  20],
        [ 30,  51,  87,  ...,  30, 248,  11],
        ...,
        [  2,   2,   2,  ...,   2,   2,   2],
        [  2,   2,   2,  ...,   2,   2,   2],
        [  2,   2,   2,  ...,   2,   2,   2]])
3
torch.Size([101, 32])
tensor([[  0,   0,   0,  ...,   0,  

In [15]:

import torch.nn.functional as F
from typing import Tuple
from torch import Tensor
class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden

In [16]:
class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)

In [17]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)

In [18]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs


In [19]:
INPUT_DIM = len(vocab)
OUTPUT_DIM = len(vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 64
# DEC_HID_DIM = 64
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)


def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters())


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 234,168 trainable parameters


In [20]:
PAD_IDX = vocab.get_stoi()['<pad>']
print(PAD_IDX)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

2


In [21]:
import math
import time
from xml.dom import IndexSizeErr


def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)
            
            
            try :
                output = model(src, trg, 0) #turn off teacher forcing
                output = output[1:].view(-1, output.shape[-1])
                trg = trg[1:].view(-1)
                loss = criterion(output, trg)
                epoch_loss += loss.item()
            except IndexError:
                epoch_loss += 6.0

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs



In [22]:

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

tensor(3.8034)
tensor(3.8728)
tensor(3.8966)
tensor(3.8900)
tensor(3.8461)
tensor(3.8030)
tensor(3.8402)
tensor(3.8518)
tensor(3.8581)
tensor(3.8942)
tensor(3.9364)
tensor(3.8332)
tensor(3.8467)
tensor(3.7304)
tensor(3.8657)
tensor(3.8294)
tensor(3.8591)
tensor(3.9461)
tensor(3.7984)
tensor(3.9274)
tensor(3.8311)
tensor(3.8309)
tensor(3.7930)
tensor(3.8659)
tensor(3.8607)
tensor(3.7834)
tensor(3.9385)
tensor(3.7227)
tensor(3.8326)
tensor(3.8697)
tensor(3.8936)
tensor(3.9279)
tensor(3.8558)
tensor(3.7682)
tensor(3.8465)
tensor(3.8076)
tensor(3.9159)
tensor(3.9159)
tensor(3.9290)
tensor(3.8520)
tensor(3.7966)
tensor(3.7377)
tensor(3.8773)
tensor(3.9467)
tensor(3.8310)
tensor(3.9085)
tensor(3.8582)
tensor(3.9527)
tensor(3.8115)
tensor(3.6687)
tensor(3.9372)
tensor(3.9138)
tensor(3.8672)
Epoch: 01 | Time: 3m 45s
	Train Loss: 3.677 | Train PPL:  39.533
	 Val. Loss: 3.894 |  Val. PPL:  49.108
